In [15]:
import numpy as np
import matplotlib
import pandas as pd
import tensorflow as tf

dataset = pd.read_csv('/Users/wzhang/Downloads/lake_erie_habs_W4_6_8_9_12_13_16_2013-2020.xlsx - HAB_data (1).csv')

train = dataset.iloc[:743,:]
test = dataset.iloc[743:,:]
print(train)
print(test)
X_train = pd.DataFrame(train.iloc[:,4:15].values)
Y_train = train.iloc[:,16:].values
print(X_train.shape)
print(Y_train.shape)
X_test = pd.DataFrame(test.iloc[:,4:15].values)
Y_test = test.iloc[:,16:].values
print(X_test.shape)
print(Y_test.shape)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



          Date  Site  Latitude (decimal deg)  Longitude (decimal deg)  \
0    5/29/2013   WE6                 41.7134                 -83.3804   
1    5/29/2013   WE8                 41.8343                 -83.3633   
2    5/29/2013   WE4                 41.8267                 -83.1930   
3    6/10/2013   WE6                 41.7144                 -83.3805   
4    6/10/2013   WE8                 41.8353                 -83.3631   
..         ...   ...                     ...                      ...   
738  10/7/2019  WE16                 41.6599                 -83.1467   
739  10/7/2019  WE13                 41.7428                 -83.1354   
740  10/7/2019  WE12                 41.7023                 -83.2639   
741  10/7/2019   WE4                 41.8270                 -83.1949   
742  6/16/2020   WE6                 41.7054                 -83.3864   

     Secchi Depth (m)  CTD Temperature (°C)  \
0                1.60                  18.5   
1                1.00        

In [16]:
from keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization, GRU, LayerNormalization, MultiHeadAttention,TimeDistributed, Input, Flatten
from sklearn.ensemble import RandomForestRegressor

In [17]:
import lightgbm as lgb
def create_lightgbm():
    params = { 
    'objective': 'regression', 
    'metric': 'rmse', 
    'boosting_type': 'gbdt', 
    'num_leaves': 31, 
    'learning_rate': 0.05, 
    'feature_fraction': 0.9, 
    }
    train_data = lgb.Dataset(X_train,label=Y_train)
    num_round = 100
    bst = lgb.train(params, train_data, num_round) 
    from sklearn.metrics import mean_squared_error as mse 
    from lightgbm import LGBMRegressor 
    
    # Create an instance of the LightGBM Regressor with the RMSE metric. 
    model = LGBMRegressor(metric='rmse') 
    
    # Train the model using the training data. 
    model.fit(X_train, Y_train) 

    res = model.predict(X_test) 
    return model


In [18]:
def create_rf():
    # Initialize Random Forest regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    rf_model.fit(X_train, Y_train)
    return rf_model




In [19]:
rf = create_rf()
lightgbm = create_lightgbm()

# ridge = create_ridge()
# svm = create_SVM()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000418 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2204
[LightGBM] [Info] Number of data points in the train set: 743, number of used features: 11
[LightGBM] [Info] Start training from score 31.734267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/basic.py:355: UserWarning: Converting column-vector to 1d array
  _log_warning("Converting column-vector to 1d array")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
import deepforest
from deepforest import CascadeForestRegressor

In [21]:
from tensorflow.keras.layers import concatenate


# gruPredict = gru.predict(X_train)
# lassoPredict = lasso.predict(X_train)
# lstmPredict = lstm.predict(X_train)
rfPredict = rf.predict(X_train)
lightgbmPredict = lightgbm.predict(X_train)
# ridgePredict = ridge.predict(X_train)
# svmPredict = svm.predict(X_train)
# rfPredict = rf.predict(X_train)
# lassoPredict.resize(671,1)
rfPredict.resize(743,1)
lightgbmPredict.resize(743,1)
# svmPredict.resize(671,1)
trainData = concatenate([rfPredict,lightgbmPredict])

In [22]:
model = CascadeForestRegressor(n_estimators = 50, criterion='squared_error', backend='sklearn')

In [23]:
model.fit(trainData, Y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[2024-07-01 19:01:24.083] Start to fit the model:
[2024-07-01 19:01:24.084] Fitting cascade layer = 0 
[2024-07-01 19:01:43.971] layer = 0  | Val MSE = 111.98793 | Elapsed = 19.887 s
[2024-07-01 19:01:44.059] Fitting cascade layer = 1 
[2024-07-01 19:05:15.356] layer = 1  | Val MSE = 127.16284 | Elapsed = 211.297 s
[2024-07-01 19:05:15.356] Early stopping counter: 1 out of 2
[2024-07-01 19:05:15.438] Fitting cascade layer = 2 
[2024-07-01 19:08:51.514] layer = 2  | Val MSE = 136.33573 | Elapsed = 216.076 s
[2024-07-01 19:08:51.514] Early stopping counter: 2 out of 2
[2024-07-01 19:08:51.514] Handling early stopping
[2024-07-01 19:08:51.563] The optimal number of layers: 1


In [27]:
# gruPredictTest = gru.predict(X_test)
# lassoPredictTest = lasso.predict(X_test)
# lstmPredictTest = lstm.predict(X_test)
rfPredictTest = rf.predict(X_test)
lightgbmPredictTest = lightgbm.predict(X_test)
# ridgePredictTest = ridge.predict(X_test)
# svmPredictTest = svm.predict(X_test)
# svmPredictTest.resize(168,1)
# svmPredictTest.resize(96,1)
# lassoPredictTest.resize(168,1)
rfPredictTest.resize(96,1)
lightgbmPredictTest.resize(96,1)
testData = concatenate([rfPredictTest, lightgbmPredictTest])

In [28]:
res = model.predict(testData)
print(res)

[2024-07-01 19:09:27.146] Start to evalute the model:
[2024-07-01 19:09:27.147] Evaluating cascade layer = 0 
[[ 5.647516  ]
 [19.195002  ]
 [ 4.214247  ]
 [ 5.1598706 ]
 [ 6.860745  ]
 [22.0379427 ]
 [ 8.8988845 ]
 [ 8.262448  ]
 [10.4966625 ]
 [23.895249  ]
 [ 3.670789  ]
 [32.205383  ]
 [11.92664943]
 [26.397375  ]
 [ 5.83340538]
 [56.0107206 ]
 [ 9.7400995 ]
 [49.10873793]
 [11.59646724]
 [33.16396375]
 [12.2465134 ]
 [58.91686543]
 [ 4.60689677]
 [ 9.59070995]
 [10.2550295 ]
 [13.93669536]
 [65.65370877]
 [19.746013  ]
 [63.44921225]
 [10.9542745 ]
 [17.8910463 ]
 [35.43149965]
 [ 1.936952  ]
 [39.690535  ]
 [19.253831  ]
 [46.68033759]
 [14.58004088]
 [ 9.95914436]
 [26.397375  ]
 [ 4.168238  ]
 [43.1299518 ]
 [25.7881475 ]
 [72.181297  ]
 [26.0986034 ]
 [17.345555  ]
 [20.06378433]
 [ 5.57526353]
 [44.85183064]
 [41.75257435]
 [83.32726272]
 [18.257258  ]
 [ 7.967056  ]
 [13.0300772 ]
 [ 8.8360455 ]
 [24.8964875 ]
 [36.70637208]
 [52.091647  ]
 [14.73867323]
 [ 7.139429  ]
 [21.

In [29]:
from sklearn.metrics import r2_score 
r2 = r2_score(Y_test, res) 
print(r2)

0.8365566870745229


In [30]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y_test,res)
print(mae)

4.694182227465503


In [31]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(Y_test,res)
print(mape)

0.40665288099531743


In [32]:
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(Y_test,res)
print(rmse)

7.40010663038554
